# Heart Rate - Sequence Prediction

This notebook is responsible to fit different RNN models like LSTM and GRU to try to model the heart rate sequences. These models would then be used to extract the features from the given heart rate time series so far

## Importing Libraries

In [2]:
# Importing scientific libarires required for analysis and handling data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

# Importing libraries related to handling of files and directory
import os
import glob
import pickle
import random

# Importing utility functions from the code base
from utils.directory_utils import *
from utils.general_utils import *
from utils.sleep_utils import *
from data_preprocessor.get_user_data import *
from clustering_utils import *
from kmeans_dm import *

# Importing Machine Learning utilities
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tslearn.clustering import TimeSeriesKMeans
from sklearn.decomposition import PCA
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import boxcox
from scipy.spatial import distance
from tslearn.metrics import dtw, cdist_dtw
from sklearn.metrics import silhouette_score
from scipy.stats import entropy

from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator

In [3]:
# First we load the data for each user seperately from their own numpy array and then stack them to get the final array
numpy_array_directory = f'../data/data_numpy_arrays/'

heart_rate_ts_data = []
calories_ts_data = []
activity_label_ts_data = []
activity_percentages = []
sleep_effeciency_ratio = []
sleep_stages_summary = []

for user_name in get_subdirectory_nms(numpy_array_directory):
#     if user_name in  ['Meghna\\']:
#         continue
    user_directory = construct_path(numpy_array_directory, user_name)

    user_heart_rate_ts_data = np.load(construct_path(user_directory, f'heart_rate_ts_data.npy'))
    user_calories_ts_data = np.load(construct_path(user_directory, f'calories_ts_data.npy'))
    user_activity_label_ts_data = np.load(construct_path(user_directory, f'activity_label_ts_data.npy'))
    user_activity_percentages = np.load(construct_path(user_directory, f'activity_percentages.npy'))
    user_sleep_effeciency_ratio = np.load(construct_path(user_directory, f'sleep_efficiency_ratio.npy'))
    user_sleep_stages_summary = pd.read_csv(construct_path(user_directory, f'sleep_stages_summary.csv'))

    heart_rate_ts_data.append(user_heart_rate_ts_data)
    calories_ts_data.append(user_calories_ts_data)
    activity_label_ts_data.append(user_activity_label_ts_data)
    activity_percentages.append(user_activity_percentages)
    sleep_effeciency_ratio.append(user_sleep_effeciency_ratio)
    sleep_stages_summary.append(user_sleep_stages_summary)

heart_rate_ts_data = np.vstack(heart_rate_ts_data)[:, :]
calories_ts_data = np.vstack(calories_ts_data)[:, :]
activity_label_ts_data = np.vstack(activity_label_ts_data)[:, :]
activity_percentages = np.vstack(activity_percentages)
sleep_effeciency_ratio = np.hstack(sleep_effeciency_ratio)
sleep_stages_summary = pd.concat(sleep_stages_summary)

In [ ]:
def generate_one_long_series(orig_data, input_shape = 15, out_shape=2):
    for day_time_series in orig_data:
        generator = TimeseriesGenerator(day_time_series, day_time_series, length=input_shape, )

In [13]:
gen = TimeseriesGenerator(heart_rate_ts_data[0], heart_rate_ts_data[0], length=15, stride=15, batch_size=1)

for i in range(len(gen)):
	x, y = gen[i]
	print('%s => %s' % (x, y))

[[90. 93. 85. 86. 89. 91. 91. 92. 91. 95. 84. 84. 88. 86. 88.]] => [88.]
[[88. 86. 78. 74. 76. 73. 77. 78. 76. 75. 81. 78. 74. 71. 75.]] => [82.]
[[82. 84. 85. 82. 85. 85. 81. 84. 86. 88. 87. 88. 84. 86. 82.]] => [79.]
[[79. 81. 84. 85. 85. 84. 85. 80. 81. 80. 81. 80. 80. 78. 81.]] => [80.]
[[80. 79. 78. 80. 85. 83. 82. 84. 87. 82. 76. 76. 83. 84. 83.]] => [85.]
[[85. 87. 84. 83. 85. 89. 84. 83. 79. 77. 79. 79. 84. 85. 82.]] => [86.]
[[86. 85. 84. 84. 92. 81. 84. 86. 80. 81. 80. 81. 91. 81. 77.]] => [84.]
[[84. 87. 83. 90. 91. 80. 75. 78. 73. 78. 78. 81. 91. 78. 81.]] => [78.]
[[ 78.  77.  96. 101.  99.  93.  96.  83.  79.  70.  78.  73.  72.  70.
   71.]] => [79.]
[[79. 73. 72. 76. 77. 78. 80. 78. 73. 75. 79. 70. 75. 69. 69.]] => [71.]
[[71. 74. 69. 68. 69. 69. 70. 72. 68. 71. 69. 72. 78. 71. 73.]] => [84.]
[[84. 71. 74. 69. 73. 73. 78. 75. 74. 70. 75. 75. 76. 82. 78.]] => [74.]
[[74. 68. 64. 65. 60. 65. 72. 78. 80. 81. 81. 75. 73. 76. 78.]] => [79.]
[[79. 80. 78. 79. 86. 84. 82. 78. 